In [3]:
# リスト 2.2.1 mecab-python3のテスト
#
# 目的:
#   ・Python から MeCab を呼び出し、（1）分かち書き（形態素の表層形を空白区切りで出力）
#     （2）品詞などの解析結果（辞書が持つ素性一式）をテキストで得る、の2パターンを確認する。
#
# 前提・理論メモ（コードの挙動は変えない）:
#   ・MeCab は入力文を「形態素」に分割し、各形態素に品詞や活用形などの素性（feature）を付与する。
#   ・`mecab-python3` は MeCab の Python バインディング。内部で Tagger を生成し、
#      - `Tagger("-Owakati")` は「分かち書き出力器」を選ぶ（形態素の表層形を空白区切りで出力するだけ）。
#      - `Tagger()` はデフォルト出力（辞書定義に依存する feature 付きフォーマット）を使う。
#   ・辞書（IPAdic / UniDic など）の違いで出力の品詞体系・feature 列構造が変わる点に注意。
#     研究で再現性を重視するなら、使用辞書を固定し（例: UniDic）、バージョンを明記すること。
#   ・`parse()` は「文字列」を返す。`-Owakati` では「トークンを空白で連結し、末尾に改行」を含む。
#     `split()` をかけると空白分割され、末尾改行は自動的に除去される（空白トークンにならない）。
#   ・詳細ノード列が必要な場合は `parseToNode()` を使うと 1 形態素ごとの構造化アクセスが可能。
#   ・環境準備の注意:
#       pip インストール例: `pip install mecab-python3 unidic-lite`
#       既存のシステム辞書を使う場合は `-d` で辞書パスを明示（例: Tagger("-d /usr/lib/mecab/dic/unidic")）。
#       ユーザー辞書を使うなら `-u user.dic` を追加。
#   ・スレッド利用時は Tagger の再利用で競合が起きることがあるため、スレッドごとに Tagger を持つのが安全。

# MecabをPythonから呼び出す
import MeCab

# 解析対象文（日本語の短文。句点「。」を含むため、文末形態素としての扱いも確認できる）
text = "これは日本語の文章です。"

# 利用パターン1: 対象文書を分かち書きにする
# - "-Owakati": 形態素（表層形）のみを空白区切りで出力するフォーマッタ
# - 返り値は末尾に改行が付いた文字列。`.split()` によってトークン列(list[str])に変換して表示している。
tagger1 = MeCab.Tagger("-Owakati")
print("【利用パターン1】分かち書き")
print(tagger1.parse(text).split())
print()

# 利用パターン2: 単語毎に分析結果（素性）を全部表示する
# - デフォルト出力は「表層形 + タブ + feature列（カンマ区切り等）」というテキスト（辞書依存）。
# - UniDic の例：表層形\t品詞-細分類\t活用型\t活用形\t原形\t読み\t発音... といった複数列。
# - 可視確認には有用だが、機械処理には `parseToNode()` で各ノードの `.feature` をパースする方が堅牢。
tagger2 = MeCab.Tagger()
print("【利用パターン2】品詞解析")
print(tagger2.parse(text))

【利用パターン1】分かち書き
['これ', 'は', '日本', '語', 'の', '文章', 'です', '。']

【利用パターン2】品詞解析
これ	コレ	コレ	此れ	代名詞			0
は	ワ	ハ	は	助詞-係助詞			
日本	ニッポン	ニッポン	日本	名詞-固有名詞-地名-国			3
語	ゴ	ゴ	語	名詞-普通名詞-一般			1
の	ノ	ノ	の	助詞-格助詞			
文章	ブンショー	ブンショウ	文章	名詞-普通名詞-一般			1
です	デス	デス	です	助動詞	助動詞-デス	終止形-一般	
。			。	補助記号-句点			
EOS

